# Lab 4 Part 2 — Transport Layer (Reliable Transport Protocol)

The goal of this lab is to introduce you to reliable transport protocols. First, you will implement a simple UDP sender and receiver as a baseline representing an unreliable transport protocol. Then, you will implement a reliable transport protocol on top of UDP.

Your protocol implementation should provide reliable data transfer between two hosts and handle network scenarios such as packet loss, packet corruption, and delayed packets. As an example, you will implement the Go-Back-N protocol. 

For delivery, submit a PDF report where you answer **only** those questions marked with **REPORT**.

## Lab Setup

Use the same setup as in Lab 4 Part 1 (run `docker-compose up -d` if needed and change the routing accordingly).

In detail, you will use:

- the "server" container as your sender.
- your "ntnu_server" as your receiver.
- the "router" container to emulate packet loss, corruption, and delay.

In [ ]:
from test_lab4_part2 import TestLab4_part2
check_progress = TestLab4_part2()
check_progress.test_1_1()

# Milestone 1 — Unreliable UDP

In this milestone, you will implement a simple UDP sender and receiver as a baseline representing an unreliable transport protocol. For simplicity, the sender reads data from a file, makes a packet for each line, and sends it to the receiver. The receiver receives the packet and writes it into a file.

## Task 1.1 — UDP Sender

You can find a skeleton code for the UDP sender in the file `udp_sender.py`. You need to complete the missing parts that are marked with `=== YOUR CODE HERE ===`.

```python

# Packet class
class Packet:
    # This class creates an abstraction for data packets.
    # We call it 'packet' and not 'segment' for consistency with the examples in the book (page 231)

(...)
  

# UDP sender class
class UDP_Sender:
    
    # This class shall handle
    # - socket creation, binding and loading data 'from above', which is a text file  (in `def __init__`)
    # - sending packets (in `def udt_send`)
    # - adding data into packets (in `def make_pkt`)
    # - the main thread that must be called to run() everything (in `def run`)
    #   This mean processing data from 'above' and send it.
    #   After there's no more data from 'above', the string 'EOT' is sent.
    #   Finally, it releases reserved resources
    
(...)    


if __name__ == '__main__':
        
    # This is the main method called by python when you run your code
    # You will have to
    # - set the necessary variables for the UDP_Sender class above
    # - create a new instance of that class
    # - start the main thread

(...)
```


## Task 1.2 — UDP Receiver

You can find a skeleton code for the UDP receiver in the file `udp_receiver.py`. You need to complete the missing parts that are marked with `=== YOUR CODE HERE ===`.

Below you can follow a general description of that file.

```python


# Packet class
class Packet:
    # This class creates an abstraction for data packets.
    # We call it 'packet' and not 'segment' for consistency with the examples in the book (page 231)

(...)

# UDP receiver class
class UDP_Receiver:
    
    # This class shall handle
    # - socket creation (in `def __init__`)
    # - receiving packets (in `def rdt_recv`)
    # - extract data from packets (in `def extract`)
    # - deliver data to the upper layer, which in our case is writing to a file (in `def deliver_data`)
    # - the main thread that must be called to run() everything (in `def run`)
    #   This mean waiting until data is received or until there is a timeout.
    #   After extraction, if the received data is the string 'EOT' it will stop and close the file
    #   If not, it will deliver the received data
    #   Finally, it releases reserved resources
    
(...)
    
    
if __name__ == '__main__':
        
    # This is the main method called by python when you run your code
    # You will have to
    # - set the necessary variables for the UDP_Receiver class above
    # - create a new instance of that class
    # - start the main thread

(...)
```


## Task 1.3 — Run UDP Sender and Receiver

- Run the "udp_receiver.py" on the "ntnu_server".
- Run the "udp_sender.py" on the "server" container.
- Check that the sender and receiver are working correctly. There should be a "data.txt" file in the "ntnu_server". The file's content should be the same as the "/home/ttm4200/work_dir/data.txt" file in the "server" container.


## Task 1.4 — Run UDP Sender and Receiver with Packet Loss

- On the "router" container, introduce 50% packet loss:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc add dev ether0 root netem loss 50%
</pre></div>

- Run your UDP sender and receiver.
- Check the content of the "data.txt" file in the "ntnu_server" and compare it with the "/home/ttm4200/work_dir/data.txt" file in the "server" container.
- Remove the packet loss:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc del dev ether0 root
</pre></div>

<span style="font-size: 25px;">Q1. </span> **REPORT**: Draw a sequence diagram of your implementation's operation under packet loss. Briefly explain your sequence diagram.

## Task 1.5 — Run UDP Sender and Receiver with Packet Corruption


- On the "router" container, introduce 50% packet corruption:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc add dev ether0 root netem corrupt 50%
</pre></div>

- Run your UDP sender and receiver.
- Check the content of the "data.txt" file in the "ntnu_server" and compare it with the "/home/ttm4200/work_dir/data.txt" file in the "server" container.

<span style="font-size: 25px;">Q2. </span> **REPORT**: Do you see corrupted lines in the "data.txt" file in the "ntnu_server"? Or do you see missing lines? Why?

<span style="font-size: 25px;">Q3. </span> **REPORT**: Draw a sequence diagram of your implementation's operation under packet corruption. Briefly explain your sequence diagram.

- Remove the packet corruption:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc del dev ether0 root
</pre></div>


# Milestone 2 — Go-Back-N Protocol

In this milestone, you will improve the reliability of your transport protocol by implementing the Go-Back-N protocol. Your implementation should follow the FSM (Finite State Machine) shown in the book (Figure 3.20 and Figure 3.21).

## Task 2.1 — GBN Sender

You can find a skeleton code for the GBN sender in the file `gbn_sender.py`. You need to complete the missing parts that are marked with `=== YOUR CODE HERE ===`. Follow the FSM and the circles in [Figure 1](#figure_1) for hints.

<a id='figure_1'></a>

|<img style="max-width: 85%; max-height: 720px" src="figures_2023/sender2.png" width="75%"/>|
|:--:|
| **Figure 1: GBN sender (Kurose, J. and Ross, K)** |

<div class="alert alert-block alert-info">
<b>Tip:</b> Open the file <code>gbn_sender.py</code> in your favourite editor and <b>read through the code first</b> while you analyse the FSM (there are comments in the code that will guide you).</div>

## Task 2.2 — GBN Receiver

You can find a skeleton code for the GBN in the file `gbn_receiver.py`. You need to complete the missing parts that are marked with `=== YOUR CODE HERE ===`. Follow the FSM and the circles in [Figure 2](#figure_2) for hints.

<a id='figure_2'></a>

|<img style="max-width: 75%; max-height: 720px" src="figures_2023/receiver2.png" />|
|:--:|
| **Figure 2: GBN receiver (Kurose, J. and Ross, K)** |

<div class="alert alert-block alert-info">
<b>Tip:</b> Open the file <code>gbn_receiver.py</code> in your favourite editor and <b>read through the code first</b> while you analyse the FSM (there are comments in the code that will guide you).</div>

## Task 2.3 — Run GBN Sender and Receiver

- Run the "gbn_receiver.py" script on the "ntnu_server".
- Run the "gbn_sender.py" script on the "server" container.
- Check that the sender and receiver are working properly. Compare the content of the "data.txt" file on the "ntnu_server" with the "/home/ttm4200/work_dir/data.txt" file on the "server" container. They should be the same.

In [ ]:
from test_lab4_part2 import TestLab4_part2
check_progress = TestLab4_part2()
check_progress.test_1_2()

## Task 2.4 — Run GBN Sender and Receiver with Packet Loss

- On the "router" container, introduce 50% packet loss:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc add dev ether0 root netem loss 50%
</pre></div>

- Run your GBN sender and receiver.
- Check the content of the "data.txt" file in the "ntnu_server" and compare it with the "/home/ttm4200/work_dir/data.txt" file in the "server" container. They should be the same.
- Remove the packet loss:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc del dev ether0 root
</pre></div>

<span style="font-size: 25px;">Q4. </span> **REPORT**: Draw a sequence diagram of your GBN implementation's operation under packet loss. Briefly explain your sequence diagram.

## Task 2.5 — Run GBN Sender and Receiver with Packet Corruption

- On the "router" container, introduce 50% packet corruption:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc add dev ether0 root netem corrupt 50%
</pre></div>

- Run your GBN sender and receiver.
- Check the content of the "data.txt" file in the "ntnu_server" and compare it with the "/home/ttm4200/work_dir/data.txt" file in the "server" container. They should be the same.
- Remove the packet corruption:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc del dev ether0 root
</pre></div>

<span style="font-size: 25px;">Q5. </span> **REPORT**: Draw a sequence diagram of your GBN implementation's operation under packet corruption. Briefly explain your sequence diagram.

<span style="font-size: 25px;">Q6. </span> **REPORT**: Does the "notcorrupt()" function in the "gbn_receiver.py" ever return "False" when running your implementation? Why or why not?


## Task 2.6 — Run GBN Sender and Receiver with Packet Delay (extra)

The `timeout` process included in the given skeleton is an approximation and has some limitations.
In this task the goal is to explore the behaviour connected to packet delay in the network and the "TIMEOUT" variable used in your "gbn_sender.py". 

- On the "router" container, introduce a delay **higher** than the "TIMEOUT" value in your "gbn_sender.py":

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc add dev ether0 root netem delay [DELAY]s
</pre></div>

- Run your GBN sender and receiver.
- Check the content of the "data.txt" file in the "ntnu_server" and compare it with the "/home/ttm4200/work_dir/data.txt" file in the "server" container. They should be the same.
- Remove the packet delay:

  <div style="background: #ffffff; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%">sudo tc qdisc del dev ether0 root
</pre></div>

<span style="font-size: 25px;">Q7. </span> **EXTRA CREDIT**: Draw a sequence diagram of your GBN implementation's operation under packet delay, explain what happens, and what are the limitations of our implementation.

## Optional Exercise 

We implemented a simple end-of-transmission (EOT) mechanism in the GBN. However, this is not a reliable mechanism. If the EOT packet is lost, the receiver will wait for the EOT packet until the timeout occurs.

<span style="font-size: 25px;">Q8. </span> **EXTRA CREDIT**: Implement a reliable EOT mechanism in the GBN. This mechanism should handle the case where the EOT packet is lost.